# Topology with 2 networks, 1 containing all the servers and the other containing the client

## Import the FABlib library 

In [83]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,656d5ba5-76e6-44f2-9693-f65937c6de5e
Bastion Host,bastion.fabric-testbed.net
Bastion Username,dayanebobo_0000167444
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Creating the experiment slice

In [84]:
slice_name = 'Topology_2'
#[site1, site2] = fablib.get_random_sites(count=2)
sites = [site1, site2] = ['DALL', 'MAX']
print(f"Sites: {site1}, {site2}")

Sites: DALL, MAX


### Creating Slice

In [85]:
slice = fablib.new_slice(name=slice_name)

#### Adding Routers

In [86]:
net1 = slice.add_l2network(name='net1', subnet=IPv4Network("192.168.1.0/24"))

router1 = slice.add_node(name='LAN-A', site=site1, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site1.lower()}-w1.fabric-testbed.net')
router1_local_iface = router1.add_component(model='NIC_Basic', name='nic_router1').get_interfaces()[0]
net1.add_interface(router1_local_iface)

In [87]:
net2 = slice.add_l2network(name='net2', subnet=IPv4Network("192.168.2.0/24"))

router2 = slice.add_node(name='LAN-B', site=site2, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site2.lower()}-w1.fabric-testbed.net')
router2_local_iface = router2.add_component(model='NIC_Basic', name='nic_router2').get_interfaces()[0]
net2.add_interface(router2_local_iface)

#### Adding Server nodes to site 1


In [88]:
site_node_count = 3
for i, site in enumerate([site1]):
    print(f"Adding nodes to {site}")
    for node_num in range(site_node_count):
        node_name = f"server_{node_num+1}"
        node = slice.add_node(name=node_name, site=site, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site.lower()}-w2.fabric-testbed.net')
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]
        network = slice.get_network(name='net1')
        network.add_interface(iface)

Adding nodes to DALL


#### Adding Client node to site 2

In [89]:
site_node_count = 1
for i, site in enumerate([site2]):
    print(f"Adding nodes to {site}")
    for node_num in range(site_node_count):
        node_name = f"client_{node_num+1}"
        node = slice.add_node(name=node_name, site=site, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site.lower()}-w2.fabric-testbed.net')
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]
        network = slice.get_network(name='net2')
        network.add_interface(iface)

Adding nodes to MAX


#### Linking Router 1 and Router 2

In [90]:
link = slice.add_l2network(name='router1_router2')
iface1 = router1.add_component(model='NIC_Basic', name=f'router1-router2').get_interfaces()[0]
link.add_interface(iface1)
iface1_dev = f"{{{{ interfaces['{iface1.get_name()}'].dev }}}}"
iface2 = router2.add_component(model='NIC_Basic', name=f'router2-router1').get_interfaces()[0]
link.add_interface(iface2)

#### Lets submit the slice now

In [ ]:
slice.submit();


Retry: 9, Time: 212 sec


ID,33c86573-1437-4f2d-8dbf-7569a098be4d
Name,Topology_2
Lease Expiration (UTC),2024-05-18 00:29:30 +0000
Lease Start (UTC),2024-05-17 00:29:32 +0000
Project ID,656d5ba5-76e6-44f2-9693-f65937c6de5e
State,Configuring


ID,Name,Site,Type,State,Error
e46b500b-b93c-4eb0-8699-01aaa3a00b7f,LAN-A,DALL,node,Active,
171e7efe-5465-4dd2-b70a-2edc93cb4bed,LAN-B,MAX,node,Active,
522cee18-0bc2-45ee-af06-3177a0b93fda,client_1,MAX,node,Active,
3058c3ea-c6f6-470b-b3ee-ee9a87c0f56a,server_1,DALL,node,Active,
31e70602-460c-41b9-8781-acacc6bbcd62,server_2,DALL,node,Active,
bd2a6bef-6eac-4759-b28f-8f197b8c25ad,server_3,DALL,node,Active,
908266e2-a3a5-4179-bbfa-b256bd91f70a,net1,DALL,network,Ticketed,
cb6524b4-bde6-4b4a-9058-abd86b8d6b3c,net2,MAX,network,Active,
9d70c210-8b55-4e19-9277-e4fb109443f1,router1_router2,,network,Active,


/opt/conda/lib/python3.11/site-packages/fabrictestbed_extensions/fablib/slice.py:204: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  slice_table.applymap(state_color)


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [82]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")